In [1]:
# Bring in all dependencies

import pandas as pd
import requests
import numpy as np
import json
import time
from datetime import date
from pprint import pprint
import sqlalchemy
#from sqlalchemy.ext.automap import automap_base
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect
from sqlalchemy import Column, Integer, String, Float
from config_poly import api_key
import sqlite3
#from splinter import Browser
#from bs4 import BeautifulSoup
#from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
# Establish link to Russell 100 index wiki page
russell_url = "https://en.wikipedia.org/wiki/Russell_1000_Index"

# Creaters browser window for scraping
#executable_path = {'executable_path': ChromeDriverManager().install()}
#browser = Browser('chrome', **executable_path, headless=False)

In [ ]:
# Visits url, creates soup object
#browser.visit(russell_url)
#soup = BeautifulSoup(browser.html, 'html.parser')

In [ ]:
thing = pd.read_html(russell_url)
company_table = thing[2]
companies = company_table["Company"][0:50]
myCompanies = pd.unique(companies[0:50])
companies

In [ ]:
tickers = company_table["Ticker"][0:50]
myTickers = tickers.drop_duplicates()
len(myTickers)

In [ ]:
# Initialize list to hold information from polygon.io
polygon_info = []
polygon_div = []

startTime = time.time()
#today = date.today()
#todays_date = today.strftime("%Y-%m-%d")

# Initializes counter to 0
count = 0

# Loops through list of ticker symbols
for ticker in tickers:
    
    # Checks if count equals 4, used to prevent exceeding 
    # the API call limit of 5 calls/minute
    if count == 4:
        # Tells program to wait a minute before running again
        time.sleep(90)
        
        # Ticker symbols to be used for API Call
        dataset_code = ticker
        
        # URLs for API Calls, the first for open and closed prices, 
        # the second for divident information
        url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
        div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"
        
        # Saving responses from both urls
        response = requests.get(url).json()
        div_response = requests.get(div_url).json()
        
        # Adding responses to info and div lists, respectively
        polygon_info.append(response)
        polygon_div.append(div_response)
        
        # Resets counter
        count = 0
        
    else:
        # Ticker symbols to be used for API Call
        dataset_code = ticker
        
        # URLs for API Calls, the first for open and closed prices, 
        # the second for divident information
        url = f"https://api.polygon.io/v1/open-close/{dataset_code}/2021-04-23?unadjusted=true&apiKey={api_key}"
        div_url = f"https://api.polygon.io/v2/reference/dividends/{dataset_code}?&apiKey={api_key}"
        
        # Saving responses from both urls
        response = requests.get(url).json()
        div_response = requests.get(div_url).json()
        
        # Adding responses to info and div lists, respectively
        polygon_info.append(response)
        polygon_div.append(div_response)
        
        # Increase counter by 2
        count += 2
#    print(response)

executionTime = (time.time() - startTime)

In [ ]:
realTime = executionTime/60
realTime

In [ ]:
company_table = pd.DataFrame({'Name':myCompanies,"Symbol":myTickers})

In [14]:
polygon_info_df = pd.DataFrame(polygon_info)
del polygon_info_df['status']
del polygon_info_df['afterHours']
del polygon_info_df['preMarket']
polygon_info_df = polygon_info_df[['symbol','from','open','high','low','close','volume']]
polygon_info_df.columns = ['symbol','date','open','high','low','close','volume']

polygon_info_df

NameError: name 'polygon_info' is not defined

In [ ]:
all_div = polygon_div[1:]
div_amt_list = []
payDate = []
ticker_list = []

for div in all_div:
    result = div['results']
    
    if result == []:
        continue
        
    else:
        ticker = div["results"][0]['ticker']
        div_amt = div["results"][0]['amount']
        pay_date = div["results"][0]['paymentDate']
    
    ticker_list.append(ticker)
    div_amt_list.append(div_amt)
    payDate.append(pay_date)
    
polygon_div_df = pd.DataFrame({"Symbol":ticker_list,"Amount ($)":div_amt_list,"Payment Date":payDate})
polygon_div_df

In [ ]:
polygon_info_df.to_csv('openClose.csv')
polygon_div_df.to_csv('dividend.csv')
company_table.to_csv('companies.csv')

In [17]:
polygon_info_df = pd.read_csv('openClose.csv')
polygon_div_df = pd.read_csv('dividend.csv')
company_table = pd.read_csv('companies.csv')

polygon_info_df.columns = ['index','symbol','date','opening','high','low','close','volume']
polygon_info_df = polygon_info_df.drop(columns='index')

In [18]:
# Create sqlite database path
database_path = "russell_stock.sqlite"

# Create an engine that can talk to the database
engine = create_engine(f"sqlite:///{database_path}")

In [10]:
# use engine to connect to existing tables/db
Base = declarative_base()
#conn = sqlite3.connect('russell_stock.sqlite')
#c = conn.cursor()

# Use this to clear out the db
# ----------------------------------
Base.metadata.drop_all(bind=engine)

# Create a "Metadata" Layer That Abstracts our SQL Database
# ----------------------------------
#Base.metadata.create_all(engine)

In [5]:
# Define our General_Info table
class Open_Close(Base):
    __tablename__ = 'openClose'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    date = Column(String)
    opening = Column(Float)
    high = Column(Float)
    low = Column(Float)
    close = Column(Float)
    volume = Column(Integer)

# Define our Dividend table
class Dividend(Base):
    __tablename__ = 'dividend'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    amount = Column(Float)
    payment_date = Column(String)

# Define our Company Table
class Company(Base):
    __tablename__ = 'company'
    __table_args__ = {'extend_existing': True}
    symbol = Column(String, primary_key=True)
    name = Column(String)

In [6]:
# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
#Open_Close = Database.classes.open_close

In [19]:
#for i in polygon_info_df.index:
#    print(polygon_info_df['from'][i])


# insert dataframe into sqlite tables
company_table.to_sql(name='company', con=engine, if_exists='replace', index=True)
polygon_div_df.to_sql(name='dividend', con=engine, if_exists='replace', index=True)
polygon_info_df.to_sql(name='openClose', con=engine, if_exists='replace', index=True)

In [23]:
# # for i in polygon_info_df.index:
#     thisFrom = polygon_info_df["date"][i]
#     thisSymbol = polygon_info_df["symbol"][i]
#     thisOpen = polygon_info_df["open"][i]
#     thisHigh = polygon_info_df["high"][i]
#    thisLow = polygon_info_df["low"][i]
#    thisClose = polygon_info_df["close"][i]
#    thisVol = polygon_info_df["volume"][i]
    
    #row = [thisFrom,thisSymbol,thisOpen,thisHigh,thisLow,thisClose,thisVol]
    
#    thisCompany = Open_Close(date=thisFrom,symbol=thisSymbol,opening=thisOpen,high=thisHigh,low=thisLow,close=thisClose,volume=thisVol)
#    session = Session(bind=engine)
    #conn.execute('Insert into open_close values(?,?,?,?,?,?,?)', row)
    #engine.execute()
#    Session.configure(bind=engine)
#    session.add(thisCompany)
    #session.commit()
    
#session.close()
    
    
    
    

In [24]:
#c.close()
#session.commit()



In [21]:
session.close()

conn.close()

engine.dispose()

NameError: name 'conn' is not defined

In [22]:
engine = create_engine(f"sqlite:///{database_path}")

connection = engine.connect()

# use engine to connect to existing tables/db
base = automap_base()
base.prepare(engine, reflect=True)

# View all of the classes/tables that automap found
thing = base.classes.keys()

print(thing)

# Save references to each table (capital because they are considered classes) 
#Company = Database.classes.company
#Dividend = Database.classes.dividend
inspector = inspect(engine)

[]
